In [46]:
def rename_id(df):
    
    new_id = "all"

    if df.level ==1:
        new_id = "Total_X_"+quants[0]+"_validation" #Total_X_0.005_validation

In [3]:

import numpy as np
import pandas as pd 
import seaborn as sns  

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
# from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import time	
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 
from pylab import rcParams

rcParams['figure.dpi'] = 120
rcParams['savefig.dpi'] = 120 
rcParams['font.size'] = 20


from tqdm import tqdm

In [4]:
df = pd.read_csv('../datasets/sales_train_validation.csv')
print(df.shape)
df.head()

(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [5]:
submission = pd.read_csv('../datasets/sample_submission.csv')
print(submission.shape)
submission.head()

(60980, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
price_df = pd.read_csv('../datasets/sell_prices.csv')
print(price_df.shape)
price_df.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [7]:
cal_df = pd.read_csv('../datasets/calendar.csv')
print(cal_df.shape)
cal_df.head(5)

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [8]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1


In [9]:
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"

In [10]:
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation


In [11]:
days_cols = df.columns[df.columns.str.contains("d_")].values
df_days = df[days_cols]
df_days

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


## step 1
### *12 LEVEL 重要*

In [12]:
unit_sales_df =pd.DataFrame()

for day in tqdm(range(1858, 1886)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
#     print("wk_id",wk_id)
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
#     print("wk_price_df",wk_price_df)
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
#     df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    unit_sales_df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:22<00:00,  1.26it/s]


In [13]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [14]:
unit_sales_df

,unit_sales_1858,unit_sales_1859,unit_sales_1860,unit_sales_1861,unit_sales_1862,unit_sales_1863,unit_sales_1864,unit_sales_1865,unit_sales_1866,unit_sales_1867,...,unit_sales_1876,unit_sales_1877,unit_sales_1878,unit_sales_1879,unit_sales_1880,unit_sales_1881,unit_sales_1882,unit_sales_1883,unit_sales_1884,unit_sales_1885
0,0.00,16.52,0.00,8.26,8.26,0.00,8.26,0.00,0.00,8.26,...,24.78,8.26,24.78,8.26,16.52,16.52,0.00,8.26,8.26,8.26
1,0.00,0.00,0.00,3.97,0.00,0.00,3.97,3.97,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,3.97,3.97,3.97,3.97
2,0.00,0.00,0.00,0.00,0.00,17.82,2.97,2.97,5.94,0.00,...,2.97,0.00,0.00,0.00,0.00,0.00,0.00,2.97,2.97,0.00
3,0.00,0.00,0.00,4.64,0.00,4.64,23.20,13.92,4.64,0.00,...,18.56,9.28,4.64,18.56,4.64,13.92,23.20,0.00,27.84,27.84
4,2.88,5.76,2.88,0.00,0.00,2.88,0.00,5.76,2.88,2.88,...,8.64,5.76,5.76,5.76,8.64,2.88,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.00,0.00,5.76,0.00,0.00,5.96,2.98,2.98,0.00,2.98,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30486,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30487,3.98,7.96,7.96,7.96,7.96,7.96,7.96,3.98,3.98,7.96,...,7.96,0.00,0.00,7.96,0.00,0.00,0.00,7.96,0.00,3.98
30488,2.56,2.56,2.56,2.56,0.00,2.56,2.56,0.00,0.00,0.00,...,1.28,1.28,1.28,1.28,0.00,2.56,1.28,1.28,2.56,5.12


In [15]:
df["dollar_sales"] = unit_sales_df.sum(axis=1)

In [16]:
print(df.shape)
print(df.columns)
df.head()

(30490, 1920)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911',
       'd_1912', 'd_1913', 'dollar_sales'],
      dtype='object', length=1920)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,231.28
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,35.73
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,44.55
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,245.92
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,89.28


In [17]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [18]:
print(df.shape)
print(df.columns)
df.head()

(30490, 1921)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912',
       'd_1913', 'dollar_sales', 'weight'],
      dtype='object', length=1921)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,231.28,0.000063
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,35.73,0.000010
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,44.55,0.000012
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,245.92,0.000067
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,89.28,0.000024


In [19]:
df.drop(columns=["dollar_sales"], inplace=True)
df["weight"]/=12
print(df.shape)
print(df.columns)
df.head()

(30490, 1920)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911',
       'd_1912', 'd_1913', 'weight'],
      dtype='object', length=1920)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,5.258191e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,8.123278e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,1.012852e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,5.591034e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,2.029797e-06


In [20]:
df.to_csv("m_df.csv")

In [21]:
df = pd.read_csv("m_df.csv",index_col=0)
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,5.258191e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,8.123278e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,1.012852e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,5.591034e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,2.029797e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,1,0,0,1,4.697087e-07
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.000000e+00
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,0,2,0,1,0,0,1,0,3.167010e-06
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,0,0,1,0,3,1,3,1.018536e-06


## step 2 
### *agg_df 12 level*

In [26]:
agg_df = pd.DataFrame(
#     df[[c for c in df.columns if c.find("d_") == 0 ]].sum()
    df[days_cols].sum()
    ).transpose()
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,41789,48362,51640,38059,37570,35343,35033,40517,48962,49795


In [27]:
id_cols=["item_id",'dept_id','cat_id','store_id','state_id']
df[id_cols]
for col in id_cols:
    agg_df[col]='all'
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1909,d_1910,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,35343,35033,40517,48962,49795,all,all,all,all,all


In [28]:
agg_df['level'] =1
agg_df['weight'] =1/12
column_order =agg_df.columns
print(column_order)
agg_df

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1911', 'd_1912', 'd_1913', 'item_id', 'dept_id', 'cat_id',
       'store_id', 'state_id', 'level', 'weight'],
      dtype='object', length=1920)


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,0.083333


In [29]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [30]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,0.083333


In [31]:
df.groupby(by=level_groupings[11]).sum() # 12级别分类

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  d_1905  d_1906  d_1907  d_1908  d_1909  d_1910  \
item_id         state_id  ...                                                   
FOODS_1_001     CA        ...       2      27       6       2       2       4   
                TX        ...       1       2       1       0       0       2   
                WI        ...       1       1       0       3       1       0   
FOODS_1_002     CA        ...       5       2       0       0       2       2   
                TX        ...       1       0       0       2       1       0   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_515 TX        ...       0       0       0       0       0       0   
                WI        ...       1       0       0       0       1       1   
HOUSEHOLD_2_516 CA        ...       0       0       0       0       0       1   
                TX        ...       1       1       0       0       1       0   
                WI        ...       0       0       0       0       0       1   

                          d_1911  d_1912  d_1913        weight  
item_id         state_id                                        
FOODS_1_001     CA             2       3       5  4.990826e-06  
                TX             0       1       0  2.342633e-06  
                WI             0      12       1  1.273170e-06  
FOODS_1_002     CA             1       2       1  1.293177e-05  
                TX             0       0       0  3.017413e-06  
...                          ...     ...     ...           ...  
HOUSEHOLD_2_515 TX             0       0       0  2.239415e-07  
                WI             1       1       0  4.478830e-07  
HOUSEHOLD_2_516 CA             1       0       1  2.295798e-06  
                TX             0       0       0  2.430845e-06  
                WI             0       0       0  6.752347e-07  

[9147 rows x 1914 columns]

In [32]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
    temp_df["level"] = level
#     temp_df["weight"] /= 12
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = "all"
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████| 10/10 [00:17<00:00,  1.76s/it]


In [33]:
print(agg_df.shape)
print(agg_df.columns)
agg_df
# agg_df.head(40)

(12350, 1920)
Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1911', 'd_1912', 'd_1913', 'item_id', 'dept_id', 'cat_id',
       'store_id', 'state_id', 'level', 'weight'],
      dtype='object', length=1920)


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,8.333333e-02
0,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,17095,21834,23187,all,all,all,all,CA,2,3.614587e-02
1,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,10615,12266,12282,all,all,all,all,TX,2,2.328074e-02
2,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,12807,14862,14326,all,all,all,all,WI,2,2.390672e-02
0,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,4387,5577,6113,all,all,all,CA_1,all,3,9.091642e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,HOUSEHOLD_2_515,all,all,all,TX,11,2.239415e-07
9143,0,0,0,0,0,0,0,0,0,0,...,1,1,0,HOUSEHOLD_2_515,all,all,all,WI,11,4.478830e-07
9144,0,2,0,1,1,1,0,2,1,2,...,1,0,1,HOUSEHOLD_2_516,all,all,all,CA,11,2.295798e-06
9145,2,1,0,0,0,0,1,1,0,0,...,0,0,0,HOUSEHOLD_2_516,all,all,all,TX,11,2.430845e-06


In [34]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])


30490 12350 42840


In [35]:
agg_df["weight"].sum()  + df["weight"].sum()


1.0000000000000007

In [36]:
agg_df.to_csv("agg_df_12level.csv")

In [37]:
agg_df=pd.read_csv("agg_df_12level.csv",index_col=0)

In [38]:
df['last28_mean']=df[[c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1]) in range(1858,1886)] + ["id"]].set_index("id").transpose().mean().reset_index()[0]

df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight,last28_mean
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,5.258191e-06,1.000000
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,8.123278e-07,0.321429
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,1.012852e-06,0.535714
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,5.591034e-06,1.892857
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,2.029797e-06,1.107143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,1,0,0,1,4.697087e-07,0.250000
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,0.000000e+00,0.000000
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,2,0,1,0,0,1,0,3.167010e-06,1.250000
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,0,0,1,0,3,1,3,1.018536e-06,1.250000


In [39]:
for l in tqdm(range(1, 10)):
    # 1. make forecast for this level
    this_level_df = agg_df[agg_df["level"]==l].copy()
    for d in range(1, 29):
        this_level_df["F_" + str(l) + "_" + str(1885 + d)] = this_level_df["d_" + str(1885 + d - 28)]
    
    # from this level, distribute forecast to all level 12 series
    # find columns with non 'all' entries
    important_column_ids = list(this_level_df[id_cols].columns[this_level_df[id_cols].nunique()!=1])
    this_level_df.reset_index(drop=True, inplace=True)
    for i, row in this_level_df.iterrows():
        if len(important_column_ids) == 0:
            level_mean_with_cond = this_level_df[[c for c in df.columns if c.find("d_")==0 and\
                                   int(c.split("_")[1]) in range(1858, 1886)]].transpose().mean()[0]
            proportion = df["last28_mean"] / level_mean_with_cond 
            for d in range(1, 29):
                df["F_" + str(l) + "_" + str(1885 + d)] = list(this_level_df["F_" + str(l) + "_" + str(1885 + d)])[0] * proportion
        else:
            cond = True
            for col in important_column_ids:
                cond = cond & (df[col] == row[col])
                
            level_mean_with_cond = this_level_df[[c for c in df.columns if c.find("d_")==0 and\
                                   int(c.split("_")[1]) in range(1858, 1886)]].transpose().mean()[i]
            proportion = df["last28_mean"] / level_mean_with_cond 
            for d in range(1, 29):
                df.loc[cond, "F_" + str(l) + "_" + str(1885 + d)] = row["F_" + str(l) + "_" + str(1885 + d)] * proportion
    


100%|██████████| 9/9 [00:26<00:00,  2.96s/it]


In [106]:
# remake agg_df
new_agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
id_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
for col in id_cols:
    new_agg_df[col] = 'all'
new_agg_df["level"] = 1
new_agg_df["weight"] = 1/12
column_order = new_agg_df.columns

for level in level_groupings:
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
    temp_df["level"] = level
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = 'all'
    new_agg_df = new_agg_df.append(temp_df[column_order])
del temp_df

In [122]:
print(agg_df.shape)
agg_df

(12350, 2190)


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,rmsse_9,wrmsse_1,wrmsse_2,wrmsse_3,wrmsse_4,wrmsse_5,wrmsse_6,wrmsse_7,wrmsse_8,wrmsse_9
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,0.520788,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02
0,14195.0,13805.0,10108.0,11047.0,9925.0,11322.0,12251.0,16610.0,14696.0,11822.0,...,0.483178,1.916994e-02,1.746490e-02,1.746490e-02,1.927312e-02,1.921862e-02,1.746490e-02,1.746490e-02,1.746490e-02,1.746490e-02
1,9438.0,9630.0,6778.0,7381.0,5912.0,9006.0,6226.0,9440.0,9376.0,7319.0,...,0.555301,1.216594e-02,1.292782e-02,1.292782e-02,1.218809e-02,1.217407e-02,1.292782e-02,1.292782e-02,1.292782e-02,1.292782e-02
2,8998.0,8314.0,6897.0,6984.0,3309.0,8883.0,9533.0,11882.0,8664.0,6431.0,...,0.862612,1.864093e-02,2.062222e-02,2.062222e-02,1.859898e-02,1.848512e-02,2.062222e-02,2.062222e-02,2.062222e-02,2.062222e-02
0,4337.0,4155.0,2816.0,3051.0,2630.0,3276.0,3450.0,5437.0,4340.0,3157.0,...,0.505354,4.915845e-03,4.479244e-03,4.594499e-03,4.959627e-03,4.940629e-03,4.517726e-03,4.503918e-03,4.594499e-03,4.594499e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.272763,6.110273e-08,6.087879e-08,6.076085e-08,6.123438e-08,6.131279e-08,6.093007e-08,6.103327e-08,6.087755e-08,6.108294e-08
9143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.832838,3.794815e-07,3.759337e-07,3.730842e-07,3.825090e-07,3.788823e-07,3.777421e-07,3.730389e-07,3.744795e-07,3.730138e-07
9144,0.0,2.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,...,0.847298,1.945106e-06,1.948903e-06,1.932755e-06,1.941667e-06,1.940085e-06,1.939742e-06,1.950521e-06,1.922305e-06,1.945226e-06
9145,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.632091,1.526383e-06,1.522722e-06,1.523291e-06,1.518293e-06,1.547561e-06,1.503088e-06,1.544312e-06,1.498467e-06,1.536514e-06


In [108]:
agg_df=new_agg_df.copy()

In [120]:
new_agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_9_1911,F_9_1912,F_9_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,45611.000000,53863.000000,46360.000000,all,all,all,all,all,1,8.333333e-02
0,14195.0,13805.0,10108.0,11047.0,9925.0,11322.0,12251.0,16610.0,14696.0,11822.0,...,18694.000000,23787.000000,21296.000000,all,all,all,all,CA,2,3.614587e-02
1,9438.0,9630.0,6778.0,7381.0,5912.0,9006.0,6226.0,9440.0,9376.0,7319.0,...,12235.000000,13595.000000,13336.000000,all,all,all,all,TX,2,2.328074e-02
2,8998.0,8314.0,6897.0,6984.0,3309.0,8883.0,9533.0,11882.0,8664.0,6431.0,...,14682.000000,16481.000000,11728.000000,all,all,all,all,WI,2,2.390672e-02
0,4337.0,4155.0,2816.0,3051.0,2630.0,3276.0,3450.0,5437.0,4340.0,3157.0,...,4904.000000,6139.000000,4669.000000,all,all,all,CA_1,all,3,9.091642e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.187041,0.232365,0.166143,HOUSEHOLD_2_515,all,all,all,TX,11,2.239415e-07
9143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.474592,0.464118,0.412273,HOUSEHOLD_2_515,all,all,all,WI,11,4.478830e-07
9144,0.0,2.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,...,0.696469,0.863845,0.799306,HOUSEHOLD_2_516,all,all,all,CA,11,2.295798e-06
9145,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.700234,0.830026,0.642661,HOUSEHOLD_2_516,all,all,all,TX,11,2.430845e-06


## step4 
### 验证算法，没啥用

In [110]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [111]:
# train_series_cols = days_cols[:-28] # X_train
# ground_truth_cols = df_days[-28:] # y_train
# forecast_cols = df.columns[df.columns.str.contains("F_")].values # y_pred


train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]

forecast_cols_dict = {}
for i in range(1, 10):
    forecast_cols_dict[i] = [c for c in df.columns if c.find("F_"+str(i)+"_") == 0]

In [112]:
np.array(df[ground_truth_cols])
# np.array(df[forecast_cols])
# df[forcast_cols_dict[1]]
np.array(df[train_series_cols])

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 6, 0, ..., 2, 0, 1],
       [0, 0, 0, ..., 1, 2, 4],
       [0, 0, 0, ..., 2, 0, 5]])

In [113]:
for i in range(1, 10):
    df["rmsse_" + str(i)] = rmsse(np.array(df[ground_truth_cols]), 
        np.array(df[forecast_cols_dict[i]]), np.array(df[train_series_cols]))
    agg_df["rmsse_" + str(i)] = rmsse(np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[forecast_cols_dict[i]]), np.array(agg_df[train_series_cols]))

In [114]:
for i in range(1, 10):
    df["wrmsse_" + str(i)] = df["weight"] * df["rmsse_" + str(i)]
    agg_df["wrmsse_" + str(i)] = agg_df["weight"] * agg_df["rmsse_" + str(i)]

In [115]:
for i in range(1, 10):
    print("Aggregation by level", str(i) + ":")
    print(df["wrmsse_" + str(i)].sum() + agg_df["wrmsse_" + str(i)].sum())
    print()

Aggregation by level 1:
0.7526969415454309

Aggregation by level 2:
0.7583278269835373

Aggregation by level 3:
0.7640636603285994

Aggregation by level 4:
0.7573295317439488

Aggregation by level 5:
0.7543085508691126

Aggregation by level 6:
0.7635048362696769

Aggregation by level 7:
0.761857669668642

Aggregation by level 8:
0.7717987726658402

Aggregation by level 9:
0.7726889521635361



In [116]:
df["last28_mean"] = df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1886, 1914)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()[0]

In [117]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,rmsse_9,wrmsse_1,wrmsse_2,wrmsse_3,wrmsse_4,wrmsse_5,wrmsse_6,wrmsse_7,wrmsse_8,wrmsse_9
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.319113,7.208131e-06,7.126347e-06,7.147485e-06,7.122012e-06,7.113091e-06,7.116438e-06,7.116090e-06,6.943477e-06,6.936151e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.494779,3.980942e-07,3.967883e-07,4.020830e-07,3.939674e-07,3.948070e-07,3.963205e-07,3.971098e-07,4.012123e-07,4.019227e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.181473,1.189901e-06,1.186452e-06,1.164533e-06,1.183363e-06,1.186457e-06,1.189809e-06,1.191102e-06,1.196448e-06,1.196657e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.670101,3.603549e-06,3.560783e-06,3.467463e-06,3.742030e-06,3.740276e-06,3.742036e-06,3.758545e-06,3.727598e-06,3.746557e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.795595,1.579326e-06,1.574731e-06,1.591477e-06,1.580985e-06,1.581618e-06,1.577931e-06,1.576314e-06,1.614455e-06,1.614896e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0.485431,2.275815e-07,2.294876e-07,2.284125e-07,2.270900e-07,2.266906e-07,2.285151e-07,2.283019e-07,2.280659e-07,2.280110e-07
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0.608944,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0.775801,2.474754e-06,2.470291e-06,2.466660e-06,2.461936e-06,2.476002e-06,2.439405e-06,2.451809e-06,2.446632e-06,2.456970e-06
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1.042014,1.048761e-06,1.078196e-06,1.063192e-06,1.042511e-06,1.036279e-06,1.079667e-06,1.074738e-06,1.065648e-06,1.061329e-06


In [118]:
level_1_mean = agg_df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1886, 1914)]].transpose().mean().reset_index()[0][0]

In [119]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,rmsse_9,wrmsse_1,wrmsse_2,wrmsse_3,wrmsse_4,wrmsse_5,wrmsse_6,wrmsse_7,wrmsse_8,wrmsse_9
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,0.520788,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02,4.339898e-02
0,14195.0,13805.0,10108.0,11047.0,9925.0,11322.0,12251.0,16610.0,14696.0,11822.0,...,0.483178,1.916994e-02,1.746490e-02,1.746490e-02,1.927312e-02,1.921862e-02,1.746490e-02,1.746490e-02,1.746490e-02,1.746490e-02
1,9438.0,9630.0,6778.0,7381.0,5912.0,9006.0,6226.0,9440.0,9376.0,7319.0,...,0.555301,1.216594e-02,1.292782e-02,1.292782e-02,1.218809e-02,1.217407e-02,1.292782e-02,1.292782e-02,1.292782e-02,1.292782e-02
2,8998.0,8314.0,6897.0,6984.0,3309.0,8883.0,9533.0,11882.0,8664.0,6431.0,...,0.862612,1.864093e-02,2.062222e-02,2.062222e-02,1.859898e-02,1.848512e-02,2.062222e-02,2.062222e-02,2.062222e-02,2.062222e-02
0,4337.0,4155.0,2816.0,3051.0,2630.0,3276.0,3450.0,5437.0,4340.0,3157.0,...,0.505354,4.915845e-03,4.479244e-03,4.594499e-03,4.959627e-03,4.940629e-03,4.517726e-03,4.503918e-03,4.594499e-03,4.594499e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.272763,6.110273e-08,6.087879e-08,6.076085e-08,6.123438e-08,6.131279e-08,6.093007e-08,6.103327e-08,6.087755e-08,6.108294e-08
9143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.832838,3.794815e-07,3.759337e-07,3.730842e-07,3.825090e-07,3.788823e-07,3.777421e-07,3.730389e-07,3.744795e-07,3.730138e-07
9144,0.0,2.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,...,0.847298,1.945106e-06,1.948903e-06,1.932755e-06,1.941667e-06,1.940085e-06,1.939742e-06,1.950521e-06,1.922305e-06,1.945226e-06
9145,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.632091,1.526383e-06,1.522722e-06,1.523291e-06,1.518293e-06,1.547561e-06,1.503088e-06,1.544312e-06,1.498467e-06,1.536514e-06


## step5 
### 输出

In [54]:
sample_sub = pd.read_csv('../datasets/sample_submission.csv')
print(sample_sub.shape)
sample_sub.head()

(60980, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
sample_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
(sample_sub['id'][:len(df)] == df['id']).all()
print(df.shape)


(30490, 2191)


In [57]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,rmsse_9,wrmsse_1,wrmsse_2,wrmsse_3,wrmsse_4,wrmsse_5,wrmsse_6,wrmsse_7,wrmsse_8,wrmsse_9
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.319113,7.208131e-06,7.126347e-06,7.147485e-06,7.122012e-06,7.113091e-06,7.116438e-06,7.116090e-06,6.943477e-06,6.936151e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.494779,3.980942e-07,3.967883e-07,4.020830e-07,3.939674e-07,3.948070e-07,3.963205e-07,3.971098e-07,4.012123e-07,4.019227e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.181473,1.189901e-06,1.186452e-06,1.164533e-06,1.183363e-06,1.186457e-06,1.189809e-06,1.191102e-06,1.196448e-06,1.196657e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.670101,3.603549e-06,3.560783e-06,3.467463e-06,3.742030e-06,3.740276e-06,3.742036e-06,3.758545e-06,3.727598e-06,3.746557e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.795595,1.579326e-06,1.574731e-06,1.591477e-06,1.580985e-06,1.581618e-06,1.577931e-06,1.576314e-06,1.614455e-06,1.614896e-06


In [58]:
sample_sub['id'][len(df):]

30490    HOBBIES_1_001_CA_1_evaluation
30491    HOBBIES_1_002_CA_1_evaluation
30492    HOBBIES_1_003_CA_1_evaluation
30493    HOBBIES_1_004_CA_1_evaluation
30494    HOBBIES_1_005_CA_1_evaluation
                     ...              
60975      FOODS_3_823_WI_3_evaluation
60976      FOODS_3_824_WI_3_evaluation
60977      FOODS_3_825_WI_3_evaluation
60978      FOODS_3_826_WI_3_evaluation
60979      FOODS_3_827_WI_3_evaluation
Name: id, Length: 30490, dtype: object

In [59]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,rmsse_9,wrmsse_1,wrmsse_2,wrmsse_3,wrmsse_4,wrmsse_5,wrmsse_6,wrmsse_7,wrmsse_8,wrmsse_9
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.319113,7.208131e-06,7.126347e-06,7.147485e-06,7.122012e-06,7.113091e-06,7.116438e-06,7.116090e-06,6.943477e-06,6.936151e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.494779,3.980942e-07,3.967883e-07,4.020830e-07,3.939674e-07,3.948070e-07,3.963205e-07,3.971098e-07,4.012123e-07,4.019227e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.181473,1.189901e-06,1.186452e-06,1.164533e-06,1.183363e-06,1.186457e-06,1.189809e-06,1.191102e-06,1.196448e-06,1.196657e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.670101,3.603549e-06,3.560783e-06,3.467463e-06,3.742030e-06,3.740276e-06,3.742036e-06,3.758545e-06,3.727598e-06,3.746557e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.795595,1.579326e-06,1.574731e-06,1.591477e-06,1.580985e-06,1.581618e-06,1.577931e-06,1.576314e-06,1.614455e-06,1.614896e-06


In [60]:
submit_df = df[["id"]]
for i in range(1, 29):
    proportion = df["last28_mean"] / level_1_mean
    submit_df["F" + str(i)] = agg_df[agg_df["level"]==1]["d_" + str(1885 + i)][0] * proportion

In [61]:
submit_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.822141,0.772321,0.738150,0.791117,1.038734,1.201515,1.287125,0.921985,0.905219,...,0.953260,1.103198,1.177973,0.868174,0.857019,0.806218,0.799147,0.924244,1.116885,1.135887
1,HOBBIES_1_002_CA_1_validation,0.060899,0.057209,0.054678,0.058601,0.076943,0.089001,0.095343,0.068295,0.067053,...,0.070612,0.081718,0.087257,0.064309,0.063483,0.059720,0.059196,0.068463,0.082732,0.084140
2,HOBBIES_1_003_CA_1_validation,0.487194,0.457672,0.437422,0.468810,0.615546,0.712009,0.762741,0.546362,0.536426,...,0.564895,0.653747,0.698058,0.514473,0.507863,0.477759,0.473569,0.547700,0.661858,0.673118
3,HOBBIES_1_004_CA_1_validation,1.552932,1.458828,1.394282,1.494333,1.962052,2.269528,2.431237,1.741528,1.709859,...,1.800602,2.083819,2.225061,1.639884,1.618814,1.522857,1.509500,1.745794,2.109671,2.145564
4,HOBBIES_1_005_CA_1_validation,1.157087,1.086970,1.038877,1.113424,1.461921,1.691021,1.811510,1.297609,1.274012,...,1.341625,1.552649,1.657889,1.221874,1.206175,1.134678,1.124725,1.300788,1.571912,1.598655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.182698,0.171627,0.164033,0.175804,0.230830,0.267003,0.286028,0.204886,0.201160,...,0.211835,0.245155,0.261772,0.192927,0.190449,0.179160,0.177588,0.205388,0.248197,0.252419
30486,FOODS_3_824_WI_3_validation,0.274047,0.257440,0.246050,0.263706,0.346245,0.400505,0.429042,0.307328,0.301740,...,0.317753,0.367733,0.392658,0.289391,0.285673,0.268739,0.266382,0.308081,0.372295,0.378629
30487,FOODS_3_825_WI_3_validation,0.761241,0.715112,0.683472,0.732516,0.961790,1.112514,1.191783,0.853690,0.838166,...,0.882648,1.021480,1.090716,0.803865,0.793536,0.746498,0.739951,0.855781,1.034153,1.051747
30488,FOODS_3_826_WI_3_validation,0.791691,0.743716,0.710811,0.761817,1.000262,1.157014,1.239454,0.887838,0.871693,...,0.917954,1.062339,1.134345,0.836019,0.825278,0.776358,0.769549,0.890013,1.075519,1.093817


In [62]:
submit_df2 = submit_df.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [63]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)

In [64]:
submit_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.822141,0.772321,0.738150,0.791117,1.038734,1.201515,1.287125,0.921985,0.905219,...,0.953260,1.103198,1.177973,0.868174,0.857019,0.806218,0.799147,0.924244,1.116885,1.135887
1,HOBBIES_1_002_CA_1_validation,0.060899,0.057209,0.054678,0.058601,0.076943,0.089001,0.095343,0.068295,0.067053,...,0.070612,0.081718,0.087257,0.064309,0.063483,0.059720,0.059196,0.068463,0.082732,0.084140
2,HOBBIES_1_003_CA_1_validation,0.487194,0.457672,0.437422,0.468810,0.615546,0.712009,0.762741,0.546362,0.536426,...,0.564895,0.653747,0.698058,0.514473,0.507863,0.477759,0.473569,0.547700,0.661858,0.673118
3,HOBBIES_1_004_CA_1_validation,1.552932,1.458828,1.394282,1.494333,1.962052,2.269528,2.431237,1.741528,1.709859,...,1.800602,2.083819,2.225061,1.639884,1.618814,1.522857,1.509500,1.745794,2.109671,2.145564
4,HOBBIES_1_005_CA_1_validation,1.157087,1.086970,1.038877,1.113424,1.461921,1.691021,1.811510,1.297609,1.274012,...,1.341625,1.552649,1.657889,1.221874,1.206175,1.134678,1.124725,1.300788,1.571912,1.598655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.182698,0.171627,0.164033,0.175804,0.230830,0.267003,0.286028,0.204886,0.201160,...,0.211835,0.245155,0.261772,0.192927,0.190449,0.179160,0.177588,0.205388,0.248197,0.252419
60976,FOODS_3_824_WI_3_evaluation,0.274047,0.257440,0.246050,0.263706,0.346245,0.400505,0.429042,0.307328,0.301740,...,0.317753,0.367733,0.392658,0.289391,0.285673,0.268739,0.266382,0.308081,0.372295,0.378629
60977,FOODS_3_825_WI_3_evaluation,0.761241,0.715112,0.683472,0.732516,0.961790,1.112514,1.191783,0.853690,0.838166,...,0.882648,1.021480,1.090716,0.803865,0.793536,0.746498,0.739951,0.855781,1.034153,1.051747
60978,FOODS_3_826_WI_3_evaluation,0.791691,0.743716,0.710811,0.761817,1.000262,1.157014,1.239454,0.887838,0.871693,...,0.917954,1.062339,1.134345,0.836019,0.825278,0.776358,0.769549,0.890013,1.075519,1.093817


In [65]:
submit_df.to_csv("submission.csv",index=False)